In [ ]:
import pandas as pd
import numpy as np
from numpy.core.umath_tests import inner1d
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from Utilities.utils import *
from Utilities.plot_functions import *

# Select Regression Model & Prediction Variable 

In [ ]:
# Select prediction variable, e.g., 'CO','aSBP','Ees'
prediction_variable = 'aSBP'  

# Select regression method, e.g., 'RF','SVR','RIDGE','GB'
regressor = 'RF'       

# Enable figure saving
save_figure_boolean = False 

experiment_type = 'insilico'    # Set the type of data being used in the analysis

# Load In Silico Data

In [ ]:
dataset = pd.read_csv('Data/insilico_data.csv') # noise is considered
dataset.head()

# correlation_matrix(dataset)

# Regression Analysis

In [ ]:
X,y = select_variable_to_predict(dataset,prediction_variable)
# save_input_output_pairs(X,y)

# X,y = load_input_output_pairs()

# - Perform k-fold cross-validation
k = 10
kf = KFold(n_splits=k); kf.get_n_splits(X); print(kf); KFold(n_splits=k, random_state=None, shuffle=False)

current_fold = 0;
y_pred_total = None; y_test_total = None;
metrics_array = np.empty((k, 4))
    
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model, y_pred = run_regressor_with_best_hyperparams(X_train, X_test,y_train, y_test,regressor,prediction_variable)
    # model, y_pred, hyper_parameters = hyperparameter_tuning(X_train, X_test, y_train, y_test,regressor)
    
    output_verbose = False
    metrics_array = calculate_metrics_for_each_fold(current_fold,y_test,y_pred,metrics_array,output_verbose)
    current_fold = current_fold + 1
    
    y_test_total, y_pred_total = accumulate_predictions(y_test, y_pred, current_fold, y_test_total, y_pred_total)
    

### Print Results

In [ ]:
calculate_regression_statistics(y_test_total, y_pred_total, metrics_array, regressor)

In [ ]:
calculate_precision_percentages(y_test_total, y_pred_total, prediction_variable)

### Plot Results

In [ ]:
plot_data(y_test_total, y_pred_total, experiment_type, prediction_variable, regressor, save_figure_boolean)